# 0. Import Library

In [1]:
import os
if not os.path.exists("./tfdet"):
    !git clone -q http://github.com/burf/tfdetection.git
    !mv ./tfdetection/tfdet ./tfdet
    !rm -rf ./tfdetection

In [2]:
#ignore warning
import warnings, os
warnings.filterwarnings(action = "ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import tfdet
#tfdet.core.util.set_seed(777) #set seed
device = tfdet.core.util.select_device(0) #set device

if True: #apply mixed precision
    tf.keras.mixed_precision.set_global_policy("mixed_float16")

# 1. Init Dataset

In [3]:
import os
path = "./balloon"
if not os.path.exists(path):
    !wget -qq --no-check-certificate http://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip
    !unzip -qq balloon_dataset.zip
print(tfdet.dataset.util.tree_dir(path))

balloon/
    train/
        605521662_a470fef77f_b.jpg
        9330497995_4cf0438cb6_k.jpg
        5178670692_63a4365c9c_b.jpg
        ...
    val/
        4838031651_3e7b5ea5c7_b.jpg
        16335852991_f55de7958d_k.jpg
        24631331976_defa3bb61f_k.jpg
        ...


In [4]:
import numpy as np

tr_path = os.path.join(path, "train/via_region_data.json")
te_path = os.path.join(path, "val/via_region_data.json")

label = tfdet.dataset.balloon.LABEL
color = None

n_class = len(label)
#image_shape = [1024, 1024]
image_shape = [512, 512]

def mask_func(mask_true):
    return np.max(mask_true, axis = 0)

def instance2semantic(x_true, y_true = None, bbox_true = None, mask_true = None, label = None):
    if y_true is not None and mask_true is not None:
        mask_true = tfdet.dataset.util.instance2semantic(y_true, mask_true, label = label)
    result = [v for v in [x_true, y_true, bbox_true, mask_true] if v is not None]
    result = result[0] if len(result) == 1 else tuple(result)
    return result

tr_dataset = tfdet.dataset.balloon.load_dataset(tr_path, mask = True,
                                                cache = None)
tr_dataset = tfdet.dataset.Dataset(tr_dataset,
                                   transform = [{"name":"load"},
                                                {"name":"resize", "image_shape":[2048, 1024], "mode":"range"},
                                                {"name":"weak_augmentation", "crop_shape":image_shape},
                                                {"name":"filter_annotation"},
                                                {"name":"label_encode", "label":label},
                                                {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                {"name":"pad", "image_shape":image_shape, "pad_val":0},
                                                {"name":"mask_downscale", "scale":4},
                                                instance2semantic, #or {"name":"load", "mask_func":mask_func},
                                               ])

te_dataset = tfdet.dataset.balloon.load_dataset(te_path, mask = True,
                                                cache = None,
                                                transform = [{"name":"load"},
                                                             {"name":"resize", "image_shape":image_shape},
                                                             {"name":"filter_annotation"},
                                                             {"name":"label_encode", "label":label},
                                                             {"name":"normalize", "mean":[123.675, 116.28, 103.53], "std":[58.395, 57.12, 57.375]},
                                                             {"name":"pad", "image_shape":image_shape, "pad_val":0},
                                                             {"name":"mask_downscale", "scale":4},
                                                             instance2semantic, #or {"name":"load", "mask_func":mask_func},
                                                            ])


tr_pipe = tfdet.dataset.PipeLoader(tr_dataset)
tr_pipe = tfdet.dataset.pipeline.args2dict(tr_pipe, keys = ["x_true", "y_true", "bbox_true", "mask_true"])
tr_pipe = tfdet.dataset.pipeline.collect(tr_pipe, keys = ["x_true", "mask_true"])

te_pipe = tfdet.dataset.PipeLoader(te_dataset)
te_pipe = tfdet.dataset.pipeline.args2dict(te_pipe, keys = ["x_true", "y_true", "bbox_true", "mask_true"])
te_pipe = tfdet.dataset.pipeline.collect(te_pipe, keys = ["x_true", "mask_true"])

Preprocessing Data: 100%|██████████| 13/13 [00:01<00:00,  6.75it/s]


# 2. Build Detector

In [5]:
with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    feature = tfdet.model.backbone.resnet50(x, weights = "imagenet_v2")
    
    out = tfdet.model.detector.upernet(feature, n_class = n_class)
    aux_out = tfdet.model.detector.aux_fcn(feature, n_class = n_class)
    model = tfdet.model.train.segmentation.train_model(x, out, aux_out)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# 3. Train

In [6]:
import shutil

epoch = 100
batch_size = 16
update_step = 64 // batch_size
save_path = "./learn/epoch@{epoch:03d}-metric@{mean_iou:.4f}-loss@{loss:.4f}.h5"

learning_rate = 1e-2
decay_rate = 1e-2 #dst learning_rate = learning_rate * decay_rate
momentum = 0.9
nesterov = True
warm_up_epoch = 5

if os.path.exists(os.path.dirname(save_path)):
    shutil.rmtree(os.path.dirname(save_path))
os.makedirs(os.path.dirname(save_path), exist_ok = True)

train_pipe = tfdet.dataset.pipeline.key_map(tr_pipe, map = {"x_true":x.name}, repeat = 8, batch_size = batch_size, shuffle = True, prefetch = True) #61 * 8 > 488
test_pipe = tfdet.dataset.pipeline.key_map(te_pipe, map = {"x_true":x.name}, batch_size = batch_size, prefetch = True)

total_step = int(train_pipe.cardinality()) #len(train_pipe)
total_test_step = int(test_pipe.cardinality()) #len(test_pipe)

ema = tfdet.callback.EMA(step = update_step, auto_apply = True, warm_up_epoch = warm_up_epoch)
metric = tfdet.callback.MeanIoU(test_pipe, label = label, name = "mean_iou")
save = tf.keras.callbacks.ModelCheckpoint(save_path, monitor = "mean_iou", mode = "max", save_best_only = True, save_weights_only = True, save_freq = "epoch", verbose = 0)
#scheduler = tfdet.callback.WarmUpCosineLearningRateScheduler(cycle = epoch, decay_rate = decay_rate, warm_up_epoch = warm_up_epoch)
scheduler = tfdet.callback.WarmUpCosineLearningRateSchedulerStep(cycle = epoch, decay_rate = decay_rate, step = update_step, total_step = total_step, warm_up_epoch = warm_up_epoch) #total step None > dynamic total step
logger = tf.keras.callbacks.CSVLogger(os.path.join(os.path.dirname(save_path), "logger.csv"), separator = ",")

In [7]:
with device:
    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum = momentum, nesterov = nesterov)
    if tf.keras.mixed_precision.global_policy().compute_dtype == "float16":
        optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
    model.compile(optimizer = optimizer)

    model.fit(train_pipe,
              #validation_data = test_pipe,
              epochs = epoch, steps_per_epoch = total_step, validation_steps = total_test_step,
              callbacks = [ema, metric, save, scheduler, logger],
              verbose = 2)

Epoch 1/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9784  |0.9575|0.9783|0.9783|
|balloon   |0.644   |0.4763|0.6453|0.6453|
+----------+--------+------+------+------+
|summary   |0.8112  |0.7169|0.8118|0.8118|
+----------+--------+------+------+------+
32/32 - 1378s - loss: 0.8188 - loss_mask: 0.5998 - loss_aux_mask: 0.2190 - ema_n_update: 32.0000 - mean_accuracy: 0.8112 - mean_iou: 0.7169 - mean_dice: 0.8118 - mean_f1: 0.8118 - learning_rate: 0.0020 - 1378s/epoch - 43s/step
Epoch 2/50

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9691  |0.9585|0.9788|0.9788|
|balloon   |0.8199  |0.5451|0.7056|0.7056|
+----------+--------+------+------+------+
|summary   |0.8945  |0.7518|0.8422|0.8422|
+----------+--------+------+------+------+
32/32 - 104s - loss: 0.4514 - loss_mask: 0.2657 - loss_aux_ma

# 4. Evaluate

In [8]:
model_path = tfdet.dataset.util.list_dir(os.path.dirname(save_path), "h5")
model_path = sorted(model_path, key = lambda x: x.split("metric@")[1].split("-")[0], reverse = True)[0]

with device:
    x = tf.keras.layers.Input(shape = [*image_shape, 3], name = "x_true")
    feature = tfdet.model.backbone.resnet50(x)

    out = tfdet.model.detector.upernet(feature, n_class = n_class)
    model = tf.keras.Model(x, out)
    model.load_weights(model_path, by_name = True)
    
metric = tfdet.core.metric.MeanIoU(label = label)
for te_dict in test_pipe:
    x_true, mask_true = te_dict["x_true"], te_dict["mask_true"]
    mask_pred = model.predict(x_true, verbose = 0)
    metric.add(mask_true, mask_pred)
print(metric.summary_text)

+----------+--------+------+------+------+
|label     |accuracy|iou   |dice  |f1    |
+----------+--------+------+------+------+
|background|0.9959  |0.9927|0.9964|0.9964|
|balloon   |0.9486  |0.8888|0.9411|0.9411|
+----------+--------+------+------+------+
|summary   |0.9723  |0.9408|0.9687|0.9687|
+----------+--------+------+------+------+


# 5. Visualize

In [ ]:
import matplotlib.pyplot as plt

out = te_dataset[0] #or next(iter(tr_dataset))
x_true, y_true, bbox_true = out[:3]
mask_true = out[3] if 3 < len(out) else None

mask_pred = model.predict(np.expand_dims(x_true, axis = 0), verbose = 0)

x_true = tfdet.dataset.transform.unnormalize(x_true, mean = [123.675, 116.28, 103.53], std = [58.395, 57.12, 57.375])

fig_size = (5, 5)
plt.figure(figsize = fig_size)
plt.imshow(x_true)
plt.figure(figsize = fig_size)
plt.imshow(mask_true[..., 0])
plt.figure(figsize = fig_size)
plt.imshow(np.argmax(mask_pred[0], axis = -1))